In [1]:
import pandas as pd

In [2]:
clv = True
NUM_THREAD = 4
STATUS = {0:"commit", 2:"abort"}
def get_thd(txn):
    return int(txn) % NUM_THREAD
def get_lock(lock_idx):
    lock_idx = int(lock_idx)
    if lock_idx == 0:
        return "SH"
    elif lock_idx == 1:
        return "EX"
    else:
        return "NONE"
def rm_txn(row, place, txn):
    thread = get_thd(txn)
    members = locktable[row][place]
    found = False
    for i, step in enumerate(members):
        if step[0] == txn:
            found = True
            break
    if not found:
        print("not found: {}".format(line))
        raise
    locktable[row][place] = members[:i] + members[i+1:]
    return thread, members[i]
def get_schedule(start, end, thread):
    return schedule.loc[start:end+1, thread][schedule.loc[start:end+1, thread].notnull()]

In [20]:
def parse(line):
    txn = ""
    tokens = line.split(" ")
    if "[thread]" in line:
        if "start running txn" in line:
            thread = int(tokens[2])
            txn = tokens[6]
            schedule.loc[ts, thread] = "start txn %s"%txn
            if clv:
                barriers[txn] = 0
        elif "finish txn" in line:
            txn = tokens[3]
            thread = get_thd(txn)
            rc = tokens[-1]
            # "[thread-%lu] finish txn %lu (aborted)\n"
            schedule.loc[ts, thread] = "thread finish txn %s %s"%(txn, thread)
    elif "[row_" in tokens[0]:
            if "add txn" in line:
                    #[row_ww] add txn 1 type 1 to owners of row 1449388428864366
                    txn = tokens[3]
                    lock = get_lock(tokens[5])
                    row = tokens[-1]
                    thread = get_thd(txn)
                    if row not in locktable:
                        locktable[row] = {}
                        locktable[row]['owners'] = []
                        locktable[row]['waiters'] = []
                        if clv:
                            locktable[row]['retired'] = []
                    locktable[row][tokens[7]].append((txn, lock))
                    if "owner" in tokens[7]:
                        schedule.loc[ts, thread] = "acquire %s lock %s"%(lock, row)
                    elif "waiter" in tokens[7]:
                        schedule.loc[ts, thread] = "wait %s lock %s"%(lock, row)
                    elif "retired" in tokens[7]:
                        schedule.loc[ts, thread] = "retire %s lock %s"%(lock, row)
            elif "rm another txn" in line:
                    # [row_clv] txn %lu rm another txn %lu from owners of row %lu
                    wounder = tokens[2]
                    txn = tokens[6]
                    row = tokens[-1]
                    place = tokens[8]
                    thread, _ = rm_txn(row, place, txn)
                    if "owner" in place:
                        schedule.loc[ts, thread] = "removed lock %s from owners by %s"%row(row, wounder)
                    elif "retired" in place:
                        schedule.loc[ts, thread] = "removed lock %s from retired by %s"%(row, wounder)
            elif "rm txn" in line:
                    txn = tokens[3]
                    row = tokens[-1]
                    thread, _ = rm_txn(row, tokens[5], txn)
                    if "owner" in tokens[5]:
                        schedule.loc[ts, thread] = "remove lock %s from owners"%row
                    elif "waiter" in tokens[5]:
                        schedule.loc[ts, thread] = "remove lock %s from waiters"%row
                    elif "retired" in tokens[5]:
                        schedule.loc[ts, thread] = "release lock %s from retired"%row
            elif "from waiter to owner" in line or "from waiters to owners" in line:
                    # [row_ww] bring 0 from waiter to owner of row 1454103086539417
                    txn = tokens[2]
                    row = tokens[-1]
                    thread = get_thd(txn)
                    found = False
                    thread, entry = rm_txn(row, "waiters", txn)
                    locktable[row]["owners"].append(entry)
                    schedule.loc[ts, thread] = "acquire lock %s"%row
            elif "from owners to retired" in line:
                    # [row_clv] move txn 0 from owners to retired type 0 of row 1459774078579136
                    txn = tokens[3]
                    row = tokens[-1]
                    thread = get_thd(txn)
                    found = False
                    lock = get_lock(tokens[-4])
                    locktable[row]["retired"].append((txn, lock))
                    schedule.loc[ts, thread] = "retire lock %s"%row
            elif "abort txn" in line:
                # [row_ww] txn 8 abort txn 6 on row 10
                wound = tokens[2]
                abort = tokens[-4]
                row = tokens[-1]
                txn = wound
                schedule.loc[ts, get_thd(wound)] = "wound %s on %s"%(abort, row)
                schedule.loc[ts, get_thd(abort)] = "%s wounded by %s"%(abort,wound)
    elif "[txn]" in tokens[0]:
        if "set txn" in line and "to abort" in line:
            wounder = tokens[1]
            txn = tokens[4]
            schedule.loc[ts, get_thd(txn)] = "txn %s set abort by %s"% (txn, wounder)
            schedule.loc[ts, get_thd(wounder)] = "wound txn %s"%txn
        elif "is set to" in line:
            # [txn] txn %lu is set to commited
            txn = tokens[2]
            status = tokens[-1]
            thread = get_thd(txn)
            schedule.loc[ts, thread] = "txn %s set %s"%(txn, status)
        elif "barrier" in line:
            # "[txn] increment barrier for txn %lu\n"
            txn = tokens[-1]
            thread = get_thd(txn)
            if tokens[1] == "increment":
                barriers[txn] += 1
            elif tokens[1] == "decrement":
                barriers[txn] -= 1
            schedule.loc[ts, thread] = "txn %s %s barrier, now should be %d"%(txn, tokens[1], barriers[txn])
        elif "# bariers" in line:
            # [txn] # bariers of txn 1 = 0
            txn = tokens[-3]
            thread = get_thd(txn)
            schedule.loc[ts, thread] = "pre-commit: txn %s's barriers = %d"%(txn, int(tokens[-1]))
    if txn in ["9"] or "txn 9" in line:
        print("[{}]: {}".format(ts, line))

In [21]:
output = open('debug.out')

In [22]:
locktable = {} # row_id: { owners: {}, waiters: {}, (opt) retired: {}}
barriers = {}
schedule = pd.DataFrame()
ts = 0

In [23]:
for line in output:
    try:
        parse(line.strip())
        ts += 1
    except:
        print(line)
        

[299]: [thread] thread 1 start running txn 9
[301]: [row_clv] add txn 9 type 0 to waiters of row 1512203540006898
[302]: [row_clv] bring 9 from waiters to owners of row 1512203540006898
[303]: [row_clv] rm txn 9 from owners of row 1512203540006898
[306]: [row_clv] move txn 9 from owners to retired type 0 of row 1512203540006898
[308]: [txn] increment barrier for txn 9
[311]: [row_clv] add txn 9 type 0 to waiters of row 1512203540016095
[314]: [row_clv] bring 9 from waiters to owners of row 1512203540016095
[320]: [row_clv] rm txn 9 from owners of row 1512203540016095
[322]: [row_clv] move txn 9 from owners to retired type 0 of row 1512203540016095
[324]: [row_clv] add txn 9 type 0 to waiters of row 1512204253987297
[326]: [txn] decrement barrier for txn 9
[328]: [row_clv] bring 9 from waiters to owners of row 1512204253987297
[329]: [txn] 11 set txn 9 to abort
[331]: [row_clv] rm txn 9 from owners of row 1512204253987297
[333]: [row_clv] txn 11 abort txn 9 on row 1512203540006898
[334]

In [32]:
import numpy as np
for i in schedule:
    print("thread {}: {}".format(i, schedule[i][schedule[i].notnull()].values[-1]))

thread 3: pre-commit: txn 39's barriers = 1
thread 1: pre-commit: txn 17's barriers = 1
thread 2: pre-commit: txn 58's barriers = 2
thread 0: release lock 1512203513655372 from retired


In [34]:
for row in locktable:
    if "39" in [item[0] for place in ['owners', 'waiters', 'retired'] for item in locktable[row][place] ]:
        print(row)
        print(locktable[row])

1512203540006898
{'owners': [], 'waiters': [], 'retired': [('11', 'EX'), ('17', 'SH'), ('20', 'EX'), ('58', 'SH'), ('39', 'SH')]}
1512203540017508
{'owners': [], 'waiters': [], 'retired': [('39', 'SH')]}
1512204296523225
{'owners': [], 'waiters': [], 'retired': [('39', 'SH')]}


In [25]:
barriers

{'3': 0,
 '1': 0,
 '2': 1,
 '0': 0,
 '7': 0,
 '5': 0,
 '4': 0,
 '9': 0,
 '11': 0,
 '13': 0,
 '8': 0,
 '12': 0,
 '6': 0,
 '16': 1,
 '15': 1,
 '17': 1,
 '10': 0,
 '14': 0,
 '18': 0,
 '22': 0,
 '26': 0,
 '30': 0,
 '34': 0,
 '38': 0,
 '42': 0,
 '46': 0,
 '50': 0,
 '54': 1,
 '19': 0,
 '23': 0,
 '27': 0,
 '31': 0,
 '35': 0,
 '20': 0,
 '58': 2,
 '39': 1}

In [35]:
get_schedule(299, 349, [1,3])

,1,3
299,start txn 9,NaN
301,wait SH lock 1512203540006898,NaN
302,acquire lock 1512203540006898,NaN
303,remove lock 1512203540006898 from owners,NaN
304,NaN,start txn 11
305,NaN,NaN
306,retire lock 1512203540006898,NaN
307,NaN,NaN
308,"txn 9 increment barrier, now should be 1",NaN
309,NaN,NaN


In [ ]:
#schedule.to_csv('sample_schedule.csv')